In [ ]:
import sys

In [ ]:
sys.path.insert(0,"../python/")
from utils import tweet_time_2_epoch

In [ ]:
from pymongo import MongoClient
import networkx as nx
import pandas as pd

# Utils

In [ ]:
def get_coll(coll_name,mongo_port=27017,mongo_db="twitter-crawler"):
    client = MongoClient('mongodb://localhost:%i/' % mongo_port)
    db = client[mongo_db]
    return db[coll_name], db

def find_some_docs(coll,sort_params=[('id',-1)],limit=10):
    res = coll.find().sort(sort_params).limit(limit)
    for item in res:
        print(item["id"],item["created_at"])

# Get collections

In [ ]:
raw_coll, db = get_coll("raw")
raw_begin_coll, _ = get_coll("raw_begining")

#### Number of documents in the collections

In [ ]:
raw_size = db.command("collstats", "raw")["count"]
raw_begin_size = db.command("collstats", "raw_begining")["count"]

In [ ]:
print("Number of docs in 'raw': %i" % raw_size)
print("Number of docs in 'raw_begining': %i" % raw_begin_size)

# Upper and lower bound of the available dataset

In [ ]:
find_some_docs(raw_begin_coll,sort_params=[("id",1)],limit=3)

In [ ]:
find_some_docs(raw_begin_coll,limit=3)

In [ ]:
find_some_docs(raw_coll,sort_params=[("id",1)],limit=3)

In [ ]:
find_some_docs(raw_coll,limit=3)

# Extract mention networks

In [ ]:
def get_mentions(coll,limit=None,use_only_tweets=True):
    res = coll.find().limit(limit) if limit != None else coll.find()
    users = {}
    edges = []
    for item in res:
        if use_only_tweets and "RT " == item['text'][:3]:
            continue
        src_id, epoch = item['user']['id_str'], int(tweet_time_2_epoch(item['created_at']))
        users[src_id] = item['user']['name']
        if 'user_mentions' in item['entities']:
            for mention in item['entities']['user_mentions']:
                trg_id = mention['id_str']
                users[trg_id] = mention['name']
                edges.append((epoch,src_id,trg_id))
    return edges, users

In [ ]:
mentions, user_names = get_mentions(raw_begin_coll)#,100000)

In [ ]:
mentions_df = pd.DataFrame(mentions,columns=["time","src","trg"])
print(len(mentions_df))

In [ ]:
user_names["42220353"]

In [ ]:
user_names["14571755"]

In [ ]:
mentions_df["src"].value_counts()[:10]

In [ ]:
mentions_df["trg"].value_counts()[:10]

In [ ]:
%matplotlib inline
mentions_df["time"].hist(bins=50)

In [ ]:
edges = []
for index, row in mentions_df.iterrows():
    edges += [(row["src"],row["trg"])]

In [ ]:
G = nx.DiGraph()

In [ ]:
G.add_edges_from(edges)

In [ ]:
N = G.number_of_nodes()
M = G.number_of_edges()
print(N,M)

In [ ]:
M / (N*(N-1))

In [ ]:
print(nx.number_weakly_connected_components(G))
#print(nx.number_strongly_connected_components(G))

In [ ]:
doc = raw_coll.find_one()

In [ ]:
doc